<a href="https://colab.research.google.com/github/LuisDavid999/UDEA-ai4eng-20251-Pruebas-Saber-Pro-Colombia/blob/main/04_preprocesado_por_regiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preprocesado**
Se tomaron las siguientes decisiones a partir de la exploración de datos para el preprocesado:

*   **Preprocesamiento para Datos Faltantes:** Este proceso abarcó la imputación de datos faltantes con valores apropiados para preservar la integridad del dataset. Al imputar con la **moda**, nos aseguramos de que no se introduzcan valores que no existan naturalmente en la distribución de la variable. Esto ayuda a mantener la forma y las características originales de la distribución de la columna, minimizando el impacto en la varianza y evitando la introducción de sesgos artificiales que podrían ocurrir con métodos que infieren valores.


*   **Eliminación de la Columna `ID`:** Esta columna, un identificador único, fue retirada al no aportar valor predictivo y para evitar la dimensionalidad innecesaria en el modelo.


*   **Eliminación de la Columna `FAMI_TIENEINTERNET.1`:** Se identificó que esta columna replica la información ya presente en la columna `FAMI_TIENEINTERNET`. Probablemente era un duplicado o una versión alternativa que no aportaba datos adicionales ni complementarios.

*   **Transformación de `PERIODO` a Categórica Ordinal:** Reconociendo la secuencia inherente de los periodos, esta columna fue convertida a un tipo de dato categórico ordinal. Esta decisión permite al modelo comprender el orden y las relaciones temporales entre los datos.

*   **Corrección de Inconsistencias Textuales en `ESTU_PRGM_ACADEMICO`:** Se realizó una limpieza en esta columna (Las otras no lo necesitan) para estandarizar formatos, corregir errores tipográficos y unificar sinónimos. Esto asegura que categorías idénticas no sean tratadas como entidades separadas por el modelo.

*   **Manejo de la Alta Cardinalidad en `ESTU_PRGM_ACADEMICO`**: Esta columna presenta un número significativamente alto de categorías únicas (programas académicos). Si se hubiera utilizado una codificación directa como One-Hot Encoding para cada programa individual, se habrían generado demasiadas columnas nuevas. Esto conduce al problema de la "maldición de la dimensionalidad", lo que puede resultar en modelos más lentos, que consumen más memoria, y son más propensos al sobreajuste (overfitting), especialmente si muchas categorías tienen muy pocos registros. Agruparlos reduce drásticamente el número de características.

*   **Escalado Min-Max(0 a 1):** Se ha implementado la técnica de escalado Min-Max (normalización) para transformar los valores de las columnas numéricas a un rango estandarizado entre 0 y 1. Ya que, muchos algoritmos de aprendizaje automático son inherentemente sensibles a la escala de las características. Si las variables tienen rangos de valores muy diferentes, la característica con el rango más amplio puede dominar la función de costo, llevando a un entrenamiento ineficiente o a resultados sesgados. El escalado Min-Max asegura que todas las características contribuyan de manera equitativa.



## **Cargar el archivo train.csv**



In [7]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder


In [8]:
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [9]:
!chmod 600 ./kaggle.json

In [10]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.04GB/s]


In [11]:
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [12]:
X_train = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")

## **Preprocesamiento para los datos faltantes**

In [13]:
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    mode_val = X_train[col].mode()[0]  # Calcula la moda
    X_train[col] = X_train[col].fillna(mode_val)  # Asigna los valores imputados

print(X_train.isna().sum())

ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_TIENELAVADORA                0
FAMI_TIENEAUTOMOVIL               0
ESTU_PRIVADO_LIBERTAD             0
ESTU_PAGOMATRICULAPROPIO          0
FAMI_TIENECOMPUTADOR              0
FAMI_TIENEINTERNET.1              0
FAMI_EDUCACIONMADRE               0
RENDIMIENTO_GLOBAL                0
coef_1                            0
coef_2                            0
coef_3                            0
coef_4                            0
dtype: int64


In [14]:
categorical_cols = X_test.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    mode_val = X_test[col].mode()[0]  # Calcula la moda
    X_test[col] = X_test[col].fillna(mode_val)  # Asigna los valores imputados

print(X_test.isna().sum())

ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_TIENELAVADORA                0
FAMI_TIENEAUTOMOVIL               0
ESTU_PRIVADO_LIBERTAD             0
ESTU_PAGOMATRICULAPROPIO          0
FAMI_TIENECOMPUTADOR              0
FAMI_TIENEINTERNET.1              0
FAMI_EDUCACIONMADRE               0
coef_1                            0
coef_2                            0
coef_3                            0
coef_4                            0
dtype: int64


## **Eliminar columna `ID` y `FAMI_TIENEINTERNET.1`**



In [15]:
X_train = X_train.drop('ID', axis=1)
X_train = X_train.drop('FAMI_TIENEINTERNET.1', axis=1)

In [16]:

X_test = X_test.drop('FAMI_TIENEINTERNET.1', axis=1)

## **Convertir la columna `PERIODO` a categórico ordinal**

In [17]:
X_train['PERIODO'] = X_train['PERIODO'].astype('category').cat.as_ordered()

print(np.unique(X_train['PERIODO']))


[20183 20184 20194 20195 20196 20202 20203 20212 20213]


In [18]:
X_test['PERIODO'] = X_test['PERIODO'].astype('category').cat.as_ordered()

print(np.unique(X_test['PERIODO']))


[20183 20184 20194 20195 20196 20202 20203 20212 20213]


In [19]:
#PERIODO
periodo_order = [20183, 20184, 20194, 20195, 20196, 20202, 20203, 20212, 20213]

X_train['PERIODO'] = pd.Categorical(X_train['PERIODO'],
                                   categories=periodo_order,
                                   ordered=True)
X_train['PERIODO'] = MinMaxScaler().fit_transform(X_train['PERIODO'].values.reshape(-1,1))
print(np.unique(X_train['PERIODO']))


[0.         0.03333333 0.36666667 0.4        0.43333333 0.63333333
 0.66666667 0.96666667 1.        ]


In [20]:
#PERIODO

X_test['PERIODO'] = pd.Categorical(X_test['PERIODO'],
                                   categories=periodo_order,
                                   ordered=True)
X_test['PERIODO'] = MinMaxScaler().fit_transform(X_test['PERIODO'].values.reshape(-1,1))
print(np.unique(X_test['PERIODO']))

[0.         0.03333333 0.36666667 0.4        0.43333333 0.63333333
 0.66666667 0.96666667 1.        ]


## **Columnas categóricas ordinales**
Se ordenan las categorías y se codifican en una escala de 0 a 1


### **Columna `ESTU_VALORMATRICULAUNIVERSIDAD`**

In [21]:
# Definir el orden
matricula_order = [
    'No pagó matrícula',
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones'
]

# Convertir a categoría ordinal
X_train['ESTU_VALORMATRICULAUNIVERSIDAD'] = pd.Categorical(
    X_train['ESTU_VALORMATRICULAUNIVERSIDAD'],
    categories=matricula_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_train['ESTU_VALORMATRICULAUNIVERSIDAD'] = X_train['ESTU_VALORMATRICULAUNIVERSIDAD'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_train['ESTU_VALORMATRICULAUNIVERSIDAD'] = scaler.fit_transform(
    X_train[['ESTU_VALORMATRICULAUNIVERSIDAD']]
)

print(X_train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts())

ESTU_VALORMATRICULAUNIVERSIDAD
0.428571    210335
0.571429    127430
0.142857     80263
0.285714     78704
0.714286     69736
1.000000     68014
0.857143     38490
0.000000     19528
Name: count, dtype: int64


In [22]:
# Convertir a categoría ordinal
X_test['ESTU_VALORMATRICULAUNIVERSIDAD'] = pd.Categorical(
    X_test['ESTU_VALORMATRICULAUNIVERSIDAD'],
    categories=matricula_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_test['ESTU_VALORMATRICULAUNIVERSIDAD'] = X_test['ESTU_VALORMATRICULAUNIVERSIDAD'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_test['ESTU_VALORMATRICULAUNIVERSIDAD'] = scaler.fit_transform(
    X_test[['ESTU_VALORMATRICULAUNIVERSIDAD']]
)

print(X_test['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts())

ESTU_VALORMATRICULAUNIVERSIDAD
0.428571    89646
0.571429    55193
0.142857    34589
0.285714    33564
0.714286    29855
1.000000    29061
0.857143    16558
0.000000     8320
Name: count, dtype: int64


### **Columna `ESTU_HORASSEMANATRABAJA`**

In [23]:
# Definir el orden
horas_semana_trabaja_order = ['0', 'Menos de 10 horas',
                   'Entre 11 y 20 horas',
                   'Entre 21 y 30 horas',
                   'Más de 30 horas']

# Convertir a categoría ordinal
X_train['ESTU_HORASSEMANATRABAJA'] = pd.Categorical(
    X_train['ESTU_HORASSEMANATRABAJA'],
    categories=horas_semana_trabaja_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_train['ESTU_HORASSEMANATRABAJA'] = X_train['ESTU_HORASSEMANATRABAJA'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_train['ESTU_HORASSEMANATRABAJA'] = scaler.fit_transform(
    X_train[['ESTU_HORASSEMANATRABAJA']]
)

print(X_train['ESTU_HORASSEMANATRABAJA'].value_counts())

ESTU_HORASSEMANATRABAJA
1.00    280209
0.00    116550
0.50    115857
0.75     92693
0.25     87191
Name: count, dtype: int64


In [24]:
# Convertir a categoría ordinal
X_test['ESTU_HORASSEMANATRABAJA'] = pd.Categorical(
    X_test['ESTU_HORASSEMANATRABAJA'],
    categories=horas_semana_trabaja_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_test['ESTU_HORASSEMANATRABAJA'] = X_test['ESTU_HORASSEMANATRABAJA'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_test['ESTU_HORASSEMANATRABAJA'] = scaler.fit_transform(
    X_test[['ESTU_HORASSEMANATRABAJA']]
)

print(X_test['ESTU_HORASSEMANATRABAJA'].value_counts())

ESTU_HORASSEMANATRABAJA
1.00    119568
0.00     50422
0.50     49386
0.75     39931
0.25     37479
Name: count, dtype: int64


### **Columna `FAMI_ESTRATOVIVIENDA`**

In [25]:
# Definir el orden y asignar valores numéricos
estratovivienda_order = ['Sin Estrato',
                          'Estrato 1',
                          'Estrato 2',
                          'Estrato 3',
                          'Estrato 4',
                          'Estrato 5',
                          'Estrato 6']

# Convertir a categoría ordinal
X_train['FAMI_ESTRATOVIVIENDA'] = pd.Categorical(
    X_train['FAMI_ESTRATOVIVIENDA'],
    categories=estratovivienda_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_train['FAMI_ESTRATOVIVIENDA'] = X_train['FAMI_ESTRATOVIVIENDA'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_train['FAMI_ESTRATOVIVIENDA'] = scaler.fit_transform(
    X_train[['FAMI_ESTRATOVIVIENDA']]
)

print(X_train['FAMI_ESTRATOVIVIENDA'].value_counts())

FAMI_ESTRATOVIVIENDA
0.333333    264808
0.500000    210685
0.166667    111991
0.666667     65514
0.833333     23608
1.000000     12605
0.000000      3289
Name: count, dtype: int64


In [26]:
# Convertir a categoría ordinal
X_test['FAMI_ESTRATOVIVIENDA'] = pd.Categorical(
    X_test['FAMI_ESTRATOVIVIENDA'],
    categories=estratovivienda_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_test['FAMI_ESTRATOVIVIENDA'] = X_test['FAMI_ESTRATOVIVIENDA'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_test['FAMI_ESTRATOVIVIENDA'] = scaler.fit_transform(
    X_test[['FAMI_ESTRATOVIVIENDA']]
)

print(X_test['FAMI_ESTRATOVIVIENDA'].value_counts())

FAMI_ESTRATOVIVIENDA
0.333333    113083
0.500000     90719
0.166667     48092
0.666667     27823
0.833333     10194
1.000000      5451
0.000000      1424
Name: count, dtype: int64


### **Columna `FAMI_EDUCACIONPADRE`**

In [27]:
# Definir el orden y asignar valores numéricos
educacion_padre_order = ['Ninguno','No sabe', 'No Aplica',
                           'Primaria incompleta', 'Primaria completa',
                           'Secundaria (Bachillerato) incompleta',
                           'Secundaria (Bachillerato) completa',
                           'Técnica o tecnológica incompleta',
                           'Técnica o tecnológica completa',
                           'Educación profesional incompleta',
                           'Educación profesional completa',
                           'Postgrado' ]

# Convertir a categoría ordinal
X_train['FAMI_EDUCACIONPADRE'] = pd.Categorical(
    X_train['FAMI_EDUCACIONPADRE'],
    categories=educacion_padre_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_train['FAMI_EDUCACIONPADRE'] = X_train['FAMI_EDUCACIONPADRE'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_train['FAMI_EDUCACIONPADRE'] = scaler.fit_transform(
    X_train[['FAMI_EDUCACIONPADRE']]
)

print(X_train['FAMI_EDUCACIONPADRE'].value_counts())

FAMI_EDUCACIONPADRE
0.545455    151467
0.272727    125675
0.909091     83117
0.454545     71654
0.727273     62995
0.363636     55958
1.000000     44169
0.818182     27084
0.636364     22552
0.000000     22008
0.090909     16592
0.181818      9229
Name: count, dtype: int64


In [28]:
# Convertir a categoría ordinal
X_test['FAMI_EDUCACIONPADRE'] = pd.Categorical(
    X_test['FAMI_EDUCACIONPADRE'],
    categories=educacion_padre_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_test['FAMI_EDUCACIONPADRE'] = X_test['FAMI_EDUCACIONPADRE'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_test['FAMI_EDUCACIONPADRE'] = scaler.fit_transform(
    X_test[['FAMI_EDUCACIONPADRE']]
)

print(X_test['FAMI_EDUCACIONPADRE'].value_counts())

FAMI_EDUCACIONPADRE
0.545455    64831
0.272727    53432
0.909091    35412
0.454545    30641
0.727273    27045
0.363636    24131
1.000000    19287
0.818182    11744
0.636364     9677
0.000000     9565
0.090909     7066
0.181818     3955
Name: count, dtype: int64


### **Columna `FAMI_EDUCACIONMADRE`**

In [29]:
# Definir el orden y asignar valores numéricos
educacion_madre_order = ['Ninguno','No sabe', 'No Aplica',
                           'Primaria incompleta', 'Primaria completa',
                           'Secundaria (Bachillerato) incompleta',
                           'Secundaria (Bachillerato) completa',
                           'Técnica o tecnológica incompleta',
                           'Técnica o tecnológica completa',
                           'Educación profesional incompleta',
                           'Educación profesional completa',
                           'Postgrado' ]

# Convertir a categoría ordinal
X_train['FAMI_EDUCACIONMADRE'] = pd.Categorical(
    X_train['FAMI_EDUCACIONMADRE'],
    categories=educacion_madre_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_train['FAMI_EDUCACIONMADRE'] = X_train['FAMI_EDUCACIONMADRE'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_train['FAMI_EDUCACIONMADRE'] = scaler.fit_transform(
    X_train[['FAMI_EDUCACIONMADRE']]
)

print(X_train['FAMI_EDUCACIONMADRE'].value_counts())

FAMI_EDUCACIONMADRE
0.545455    165408
0.272727     99420
0.727273     89542
0.909091     85326
0.454545     81012
0.363636     56125
1.000000     46246
0.636364     27533
0.818182     22470
0.000000     14483
0.090909      3017
0.181818      1918
Name: count, dtype: int64


In [30]:
# Convertir a categoría ordinal
X_test['FAMI_EDUCACIONMADRE'] = pd.Categorical(
    X_test['FAMI_EDUCACIONMADRE'],
    categories=educacion_madre_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_test['FAMI_EDUCACIONMADRE'] = X_test['FAMI_EDUCACIONMADRE'].cat.codes

# Escalar entre 0 y 1
scaler = MinMaxScaler()
X_test['FAMI_EDUCACIONMADRE'] = scaler.fit_transform(
    X_test[['FAMI_EDUCACIONMADRE']]
)

print(X_test['FAMI_EDUCACIONMADRE'].value_counts())

FAMI_EDUCACIONMADRE
0.545455    70872
0.272727    42149
0.727273    38571
0.909091    36477
0.454545    34566
0.363636    24399
1.000000    20049
0.636364    11650
0.818182     9647
0.000000     6335
0.090909     1286
0.181818      785
Name: count, dtype: int64


### **Columna Objetivo `RENDIMIENTO_GLOBAL`**

In [31]:

# Definir el orden y asignar valores numéricos
rendimiento_global_order = ['bajo',
                          'medio-bajo',
                          'medio-alto',
                          'alto']

# Convertir a categoría ordinal
X_train['RENDIMIENTO_GLOBAL'] = pd.Categorical(
    X_train['RENDIMIENTO_GLOBAL'],
    categories=rendimiento_global_order,
    ordered=True
)

# Obtener códigos numéricos (0 a n-1)
X_train['RENDIMIENTO_GLOBAL'] = X_train['RENDIMIENTO_GLOBAL'].cat.codes


print(X_train['RENDIMIENTO_GLOBAL'].value_counts())

RENDIMIENTO_GLOBAL
3    175619
0    172987
1    172275
2    171619
Name: count, dtype: int64


## **Columnas categóricas nominales binarias**
Se determinan los dos posibles valores y como están escritos y se codifican como 1 o 0

In [32]:

binary_cols = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
               'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']
X_train[binary_cols] = X_train[binary_cols].apply(lambda x: x.map({'Si': 1, 'No': 0, 'S': 1, 'N': 0}))

X_train.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,0.966667,ENFERMERIA,BOGOTÁ,0.857143,0.25,0.500000,1,0.636364,1,1,0,0,1,1.000000,2,0.322,0.208,0.310,0.267
1,0.966667,DERECHO,ATLANTICO,0.571429,0.00,0.500000,0,0.727273,1,0,0,0,1,0.636364,0,0.311,0.215,0.292,0.264
2,0.666667,MERCADEO Y PUBLICIDAD,BOGOTÁ,0.571429,1.00,0.500000,1,0.545455,1,0,0,0,0,0.545455,0,0.297,0.214,0.305,0.264
3,0.400000,ADMINISTRACION DE EMPRESAS,SANTANDER,0.714286,0.00,0.666667,1,0.090909,1,0,0,0,1,0.545455,3,0.485,0.172,0.252,0.190
4,0.966667,PSICOLOGIA,ANTIOQUIA,0.571429,0.75,0.500000,1,0.363636,1,1,0,0,1,0.363636,1,0.316,0.232,0.285,0.294


In [33]:
X_test[binary_cols] = X_test[binary_cols].apply(lambda x: x.map({'Si': 1, 'No': 0, 'S': 1, 'N': 0}))

X_test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,550236,0.000000,TRABAJO SOCIAL,BOLIVAR,0.142857,0.25,0.500000,1,0.727273,1,0,0,1,1,0.363636,0.328,0.219,0.317,0.247
1,98545,0.666667,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,0.571429,0.75,0.333333,1,0.545455,1,0,0,0,1,0.727273,0.227,0.283,0.296,0.324
2,499179,0.966667,INGENIERIA MECATRONICA,BOGOTÁ,0.428571,0.00,0.500000,1,0.454545,1,0,0,0,1,0.545455,0.285,0.228,0.294,0.247
3,782980,0.400000,CONTADURIA PUBLICA,SUCRE,0.428571,0.75,0.166667,0,0.272727,1,0,0,0,0,0.272727,0.160,0.408,0.217,0.294
4,785185,0.966667,ADMINISTRACION DE EMPRESAS,ATLANTICO,0.571429,0.50,0.333333,1,0.545455,1,0,0,0,1,0.545455,0.209,0.283,0.306,0.286


## **Normalización de textos en la columna `ESTU_PRGM_ACADEMICO`**

In [34]:
# Normalizar textos (mayúsculas, tildes, espacios)
X_train['ESTU_PRGM_ACADEMICO'] = X_train['ESTU_PRGM_ACADEMICO'].str.strip().str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [35]:
# Normalizar textos (mayúsculas, tildes, espacios)
X_test['ESTU_PRGM_ACADEMICO'] = X_test['ESTU_PRGM_ACADEMICO'].str.strip().str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

## **Manejo de la Alta Cardinalidad**
Se agruparon los programas en áreas de conocimiento para reducir el número de categorías.

In [36]:
# Esta variable tiene muchas categorías (948), se agrupan para reducirlas:
def agrupar_programas(programa):
    programa = programa.lower()
    if any(word in programa for word in ['admin', 'nego', 'mercad', 'finanz',]):
        return 'Administración y Negocios'
    elif any(word in programa for word in ['ingenier', 'tecnolog', 'compu']):
        return 'Ingeniería y Tecnología'
    elif any(word in programa for word in ['medicina', 'enfermer', 'salud', 'farmacia','quiru','tera']):
        return 'Ciencias de la Salud'
    elif any(word in programa for word in ['derecho', 'jurisprudencia', 'politica','poli','cultu',]):
        return 'Derecho y Ciencias Políticas'
    elif any(word in programa for word in ['educacion', 'pedagog', 'licenciatura']):
        return 'Educación'
    elif any(word in programa for word in ['arte', 'diseño', 'musica', 'cine','foto','graf','audio']):
        return 'Artes y Diseño'
    elif any(word in programa for word in ['ciencia', 'matem', 'fisica', 'quimic', 'biolog','geo','estad','astro','micro','ocea']):
        return 'Ciencias Básicas'
    elif any(word in programa for word in ['cons', 'arquite', 'fisica', 'quimic', 'biolog','geo']):
        return 'Construcción'
    elif any(word in programa for word in ['ling', 'lite', 'lengu', 'espa', 'filol','idio','letr','tradu']):
        return 'Lenguaje'
    elif any(word in programa for word in ['deport', 'activi', 'entren', 'fisio' ]):
        return 'Actividad Física y Deportes'
    else:
        return 'Otras'

X_train['AREA_CONOCIMIENTO'] = X_train['ESTU_PRGM_ACADEMICO'].apply(agrupar_programas)

In [37]:
X_test['AREA_CONOCIMIENTO'] = X_test['ESTU_PRGM_ACADEMICO'].apply(agrupar_programas)

## **Codificador One-Hot para `AREA_CONOCIMIENTO`**

In [38]:

## 1. Crear el codificador one-hot
onehot_encoder = OneHotEncoder(sparse_output=False, dtype='int8')

## 2. Ajustar y transformar la columna
areas_encoded = onehot_encoder.fit_transform(
    X_train[['AREA_CONOCIMIENTO']]
)

## 3. Obtener los nombres de las nuevas columnas
nombres_columnas = onehot_encoder.get_feature_names_out(
    input_features=['AREA_CONOCIMIENTO']
)

## 4. Crear DataFrame con las nuevas columnas
areas_df = pd.DataFrame(
    areas_encoded,
    columns=nombres_columnas,
    index=X_train.index
)

## 5. Concatenar con el X_train original
X_train = pd.concat([X_train, areas_df], axis=1)

## 6. Eliminar la columna original
X_train.drop('ESTU_PRGM_ACADEMICO', axis=1, inplace=True)
X_train.drop('AREA_CONOCIMIENTO', axis=1, inplace=True)

In [39]:
## 1. Crear el codificador one-hot
onehot_encoder = OneHotEncoder(sparse_output=False, dtype='int8')

## 2. Ajustar y transformar la columna
areas_encoded = onehot_encoder.fit_transform(
    X_test[['AREA_CONOCIMIENTO']]
)

## 3. Obtener los nombres de las nuevas columnas
nombres_columnas = onehot_encoder.get_feature_names_out(
    input_features=['AREA_CONOCIMIENTO']
)

## 4. Crear DataFrame con las nuevas columnas
areas_df = pd.DataFrame(
    areas_encoded,
    columns=nombres_columnas,
    index=X_test.index
)

## 5. Concatenar con el X_test original
X_test = pd.concat([X_test, areas_df], axis=1)

## 6. Eliminar la columna original
X_test.drop('ESTU_PRGM_ACADEMICO', axis=1, inplace=True)
X_test.drop('AREA_CONOCIMIENTO', axis=1, inplace=True)

## **Manejo de la Alta Cardinalidad**
Se agruparon los departamentos en regiones para reducir el número de categorías.

In [40]:
# Esta variable tiene muchas categorías (32), se agrupan para reducirlas:
def agrupar_departamentos(departamento):

    if any(word in departamento for word in ['VALLE', 'CHOCO', 'CAUCA', 'NARIÑO',]):
        return 'PACIFICA'
    elif any(word in departamento for word in ['ARAUCA', 'CASANARE', 'META','VICHADA']):
        return 'ORINOQUIA'
    elif any(word in departamento for word in ['PUTUMAYO', 'AMAZONAS', 'CAQUETA', 'VAUPES','GUAVIARE']):
        return 'AMAZONIA'
    elif any(word in departamento for word in ['ATLANTICO', 'BOLIVAR', 'CESAR','CORDOBA','LA GUAJIRA','MAGDALENA','SUCRE','SAN ANDRES']):
        return 'CARIBE'
    else:
        return 'ANDINA'

X_train['ESTU_PRGM_REGION'] = X_train['ESTU_PRGM_DEPARTAMENTO'].apply(agrupar_departamentos)
X_test['ESTU_PRGM_REGION'] = X_test['ESTU_PRGM_DEPARTAMENTO'].apply(agrupar_departamentos)

## **Codificador One-Hot para `ESTU_PRGM_DEPARTAMENTO`**

In [41]:
## 1. Crear el codificador one-hot
onehot_encoder2 = OneHotEncoder(sparse_output=False, dtype='int8')

## 2. Ajustar y transformar la columna
departamentos_encoded = onehot_encoder2.fit_transform(
    X_train[['ESTU_PRGM_REGION']]
)

## 3. Obtener los nombres de las nuevas columnas
nombres_columnas2 = onehot_encoder2.get_feature_names_out(
    input_features=['ESTU_PRGM_REGION']
)

## 4. Crear DataFrame con las nuevas columnas
departamentos_df = pd.DataFrame(
    departamentos_encoded,
    columns=nombres_columnas2,
    index=X_train.index
)

## 5. Concatenar con el X_train original
X_train = pd.concat([X_train, departamentos_df], axis=1)

## 6. Eliminar la columna original
X_train.drop('ESTU_PRGM_DEPARTAMENTO', axis=1, inplace=True)
X_train.drop('ESTU_PRGM_REGION', axis=1, inplace=True)

In [42]:
## 1. Crear el codificador one-hot
onehot_encoder2 = OneHotEncoder(sparse_output=False, dtype='int8')

## 2. Ajustar y transformar la columna
departamentos_encoded = onehot_encoder2.fit_transform(
    X_test[['ESTU_PRGM_REGION']]
)

## 3. Obtener los nombres de las nuevas columnas
nombres_columnas2 = onehot_encoder2.get_feature_names_out(
    input_features=['ESTU_PRGM_REGION']
)

## 4. Crear DataFrame con las nuevas columnas
departamentos_df = pd.DataFrame(
    departamentos_encoded,
    columns=nombres_columnas2,
    index=X_test.index
)

## 5. Concatenar con el X_test original
X_test = pd.concat([X_test, departamentos_df], axis=1)

## 6. Eliminar la columna original
X_test.drop('ESTU_PRGM_DEPARTAMENTO', axis=1, inplace=True)
X_test.drop('ESTU_PRGM_REGION', axis=1, inplace=True)

In [43]:
# Configurar para mostrar todas las columnas
pd.set_option('display.max_columns', None)
# Mostrar las primeras filas con todas las columnas
X_train.head()

,PERIODO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4,AREA_CONOCIMIENTO_Actividad Física y Deportes,AREA_CONOCIMIENTO_Administración y Negocios,AREA_CONOCIMIENTO_Artes y Diseño,AREA_CONOCIMIENTO_Ciencias Básicas,AREA_CONOCIMIENTO_Ciencias de la Salud,AREA_CONOCIMIENTO_Construcción,AREA_CONOCIMIENTO_Derecho y Ciencias Políticas,AREA_CONOCIMIENTO_Educación,AREA_CONOCIMIENTO_Ingeniería y Tecnología,AREA_CONOCIMIENTO_Lenguaje,AREA_CONOCIMIENTO_Otras,ESTU_PRGM_REGION_AMAZONIA,ESTU_PRGM_REGION_ANDINA,ESTU_PRGM_REGION_CARIBE,ESTU_PRGM_REGION_ORINOQUIA,ESTU_PRGM_REGION_PACIFICA
0,0.966667,0.857143,0.25,0.500000,1,0.636364,1,1,0,0,1,1.000000,2,0.322,0.208,0.310,0.267,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,0.966667,0.571429,0.00,0.500000,0,0.727273,1,0,0,0,1,0.636364,0,0.311,0.215,0.292,0.264,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,0.666667,0.571429,1.00,0.500000,1,0.545455,1,0,0,0,0,0.545455,0,0.297,0.214,0.305,0.264,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0.400000,0.714286,0.00,0.666667,1,0.090909,1,0,0,0,1,0.545455,3,0.485,0.172,0.252,0.190,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0.966667,0.571429,0.75,0.500000,1,0.363636,1,1,0,0,1,0.363636,1,0.316,0.232,0.285,0.294,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0


In [44]:
# Configurar para mostrar todas las columnas
pd.set_option('display.max_columns', None)
# Mostrar las primeras filas con todas las columnas
X_test.head()

,ID,PERIODO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4,AREA_CONOCIMIENTO_Actividad Física y Deportes,AREA_CONOCIMIENTO_Administración y Negocios,AREA_CONOCIMIENTO_Artes y Diseño,AREA_CONOCIMIENTO_Ciencias Básicas,AREA_CONOCIMIENTO_Ciencias de la Salud,AREA_CONOCIMIENTO_Construcción,AREA_CONOCIMIENTO_Derecho y Ciencias Políticas,AREA_CONOCIMIENTO_Educación,AREA_CONOCIMIENTO_Ingeniería y Tecnología,AREA_CONOCIMIENTO_Lenguaje,AREA_CONOCIMIENTO_Otras,ESTU_PRGM_REGION_AMAZONIA,ESTU_PRGM_REGION_ANDINA,ESTU_PRGM_REGION_CARIBE,ESTU_PRGM_REGION_ORINOQUIA,ESTU_PRGM_REGION_PACIFICA
0,550236,0.000000,0.142857,0.25,0.500000,1,0.727273,1,0,0,1,1,0.363636,0.328,0.219,0.317,0.247,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,98545,0.666667,0.571429,0.75,0.333333,1,0.545455,1,0,0,0,1,0.727273,0.227,0.283,0.296,0.324,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,499179,0.966667,0.428571,0.00,0.500000,1,0.454545,1,0,0,0,1,0.545455,0.285,0.228,0.294,0.247,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,782980,0.400000,0.428571,0.75,0.166667,0,0.272727,1,0,0,0,0,0.272727,0.160,0.408,0.217,0.294,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,785185,0.966667,0.571429,0.50,0.333333,1,0.545455,1,0,0,0,1,0.545455,0.209,0.283,0.306,0.286,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [45]:
archivo = X_train.to_csv("trainregiones.csv", index = False)
archivo = X_test.to_csv("testregiones.csv", index = False)

In [46]:
!head train01.csv
!head test01.csv

head: cannot open 'train01.csv' for reading: No such file or directory
head: cannot open 'test01.csv' for reading: No such file or directory
